In [1]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

def fetch_monthly_hourly(symbol, year, month):
    start = pd.Timestamp(f"{year}-{month:02d}-01")
    end = (start + pd.offsets.MonthBegin(1))

    start_ts = int(start.timestamp() * 1000)
    end_ts = int(end.timestamp() * 1000)

    all_data = []
    while start_ts < end_ts:
        params = {
            "symbol": symbol,
            "interval": "1h",
            "startTime": start_ts,
            "limit": 1000
        }
        response = requests.get("https://api.binance.com/api/v3/klines", params=params)
        data = response.json()
        if not data:
            break

        all_data.extend(data)
        start_ts = data[-1][0] + 1
        time.sleep(0.25)

    df = pd.DataFrame(all_data, columns=["timestamp", "open", "high", "low", "close", "volume", "_close_time", "_quote_asset_volume", "_num_trades", "_taker_buy_base_vol", "_taker_buy_quote_vol", "_ignore"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df = df[["timestamp", "open", "high", "low", "close", "volume"]]
    df[["open", "high", "low", "close", "volume"]] = df[["open", "high", "low", "close", "volume"]].astype(float)
    return df


def getDataForPeriod(start, end, ticker):
    all_dfs = []

    for year in [start, end]:
        start_month = 1
        end_month = 12

        if year == 2024:
            start_month = 1
        if year == 2025:
            end_month = 5

        for month in range(start_month, end_month + 1):
            print(f"Fetching {ticker} {year}-{month:02d}")
            df = fetch_monthly_hourly(ticker, year, month)
            all_dfs.append(df)

    return pd.concat(all_dfs)


def resample(df, interval="4h", column_map=None):
    df.set_index("timestamp", inplace=True)
    if column_map is None:
        column_map = {
            "open": "first",
            "high": "max",
            "low": "min",
            "close": "last",
            "volume": "sum"
        }
    resampled = df.resample(interval).agg(column_map).dropna()
    resampled.reset_index(inplace=True)
    df.reset_index(inplace=True)
    return resampled

In [2]:
btc_1h = getDataForPeriod(2024, 2025, 'BTCUSDT')
btc_1h.drop_duplicates(inplace=True)
btc_4h = resample(btc_1h, interval="4h")
btc_1d = resample(btc_1h, interval="1d")
btc_1w = resample(btc_1h, interval="1W")

Fetching BTCUSDT 2024-01
Fetching BTCUSDT 2024-02
Fetching BTCUSDT 2024-03
Fetching BTCUSDT 2024-04
Fetching BTCUSDT 2024-05
Fetching BTCUSDT 2024-06
Fetching BTCUSDT 2024-07
Fetching BTCUSDT 2024-08
Fetching BTCUSDT 2024-09
Fetching BTCUSDT 2024-10
Fetching BTCUSDT 2024-11
Fetching BTCUSDT 2024-12
Fetching BTCUSDT 2025-01
Fetching BTCUSDT 2025-02
Fetching BTCUSDT 2025-03
Fetching BTCUSDT 2025-04
Fetching BTCUSDT 2025-05


In [3]:
eth_1h = getDataForPeriod(2024, 2025, 'ETHUSDT')
eth_1h.drop_duplicates(inplace=True)
eth_4h = resample(eth_1h, interval="4h")
eth_1d = resample(eth_1h, interval="1d")
eth_1w = resample(eth_1h, interval="1W")

Fetching ETHUSDT 2024-01
Fetching ETHUSDT 2024-02
Fetching ETHUSDT 2024-03
Fetching ETHUSDT 2024-04
Fetching ETHUSDT 2024-05
Fetching ETHUSDT 2024-06
Fetching ETHUSDT 2024-07
Fetching ETHUSDT 2024-08
Fetching ETHUSDT 2024-09
Fetching ETHUSDT 2024-10
Fetching ETHUSDT 2024-11
Fetching ETHUSDT 2024-12
Fetching ETHUSDT 2025-01
Fetching ETHUSDT 2025-02
Fetching ETHUSDT 2025-03
Fetching ETHUSDT 2025-04
Fetching ETHUSDT 2025-05


In [4]:
import time

# Helper function that fetches the data.
def getStockData(API_KEY, symbol, function, market=None, outputsize="full", interval=None, month=None, count=0):
    symbolString = ""
    intervalString = ""
    outputsizeString = ""
    monthString = ""
    marketString = ""

    count = checkRequests(count)
    
    if symbol != None:
        symbolString = f"&symbol={symbol}" 
    if interval != None:
        intervalString = f"&interval={interval}" 
    if outputsize != None:
        outputsizeString = f"&outputsize={outputsize}" 
    if month != None:
        monthString = f"&month={month}"
    if market != None:
        marketString = f"&market={market}"

    URL = f"https://www.alphavantage.co/query?function={function}{symbolString}{marketString}{intervalString}{outputsizeString}{monthString}&apikey={API_KEY}"
    response = requests.get(URL)
    data = response.json()
    if data == None:
        print("None")
    data = data.get(f"Time Series ({interval})", {})
    df = pd.DataFrame.from_dict(data, orient="index")
    df.insert(0, "timestamp", df.index)
    return df, count

# Gets stock data for a a period of years specified by starting year and ending year.
def getDataForPeriod(API_KEY, symbol, function, interval, yearStart, yearEnd, count, market=None):
    dfList = []
    for year in range(yearStart, yearEnd+1):
        print(f"Getting {symbol} data for {year}")
        for month in range(1, 13):
            month = f"{year}-{str(month).zfill(2)}"
            df, count = getStockData(API_KEY=API_KEY, symbol=symbol, function=function, interval=interval, month=month, count=count, market=market)
            dfList.append(df)
    full_df = pd.concat(dfList, ignore_index=True)
    return full_df, count

# Checks if 75 requests has been made and sets a 1 minute timer before requesting again.
def checkRequests(count):
    if count == 75:
        print("API requests reached for min... Waiting")
        time.sleep(62)
        count = 0
    count += 1
    return count

In [5]:
def prepare(df):
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df = df.sort_values("timestamp").set_index("timestamp", drop=False)
    cols = ["1. open", "2. high", "3. low", "4. close", "5. volume"]
    df[cols] = df[cols].apply(pd.to_numeric)
    rename_map = {"1. open": "open",                                            # Map for renaming columns to standard
                  "2. high": "high", 
                  "3. low": "low",
                  "4. close": "close", 
                  "5. volume": "volume", 
                  "date": "timestamp",
                  "volume usdt": "volume",
                  }
    
    df.rename(columns=rename_map, inplace=True)
    return df

In [6]:
API_KEY = "5HUC90FRQ4H9PK0Q"

count = 0

aapl_1h, count = getDataForPeriod(API_KEY=API_KEY, symbol="AAPL", function="TIME_SERIES_INTRADAY", interval="60min", yearStart=2024, yearEnd=2025, count=count)
aapl_1h.drop_duplicates(inplace=True)
aapl_1h = prepare(aapl_1h)
aapl_4h = resample(aapl_1h, interval="4h")
aapl_1d = resample(aapl_1h, interval="1d")
aapl_1w = resample(aapl_1h, interval="1W")

tsla_1h, count = getDataForPeriod(API_KEY=API_KEY, symbol="TSLA", function="TIME_SERIES_INTRADAY", interval="60min", yearStart=2024, yearEnd=2025, count=count)
tsla_1h.drop_duplicates(inplace=True)
tsla_1h = prepare(tsla_1h)
tsla_4h = resample(tsla_1h, interval="4h")
tsla_1d = resample(tsla_1h, interval="1d")
tsla_1w = resample(tsla_1h, interval="1W")

amzn_1h, count = getDataForPeriod(API_KEY=API_KEY, symbol="AMZN", function="TIME_SERIES_INTRADAY", interval="60min", yearStart=2024, yearEnd=2025, count=count)
amzn_1h.drop_duplicates(inplace=True)
amzn_1h = prepare(amzn_1h)
amzn_4h = resample(amzn_1h, interval="4h")
amzn_1d = resample(amzn_1h, interval="1d")
amzn_1w = resample(amzn_1h, interval="1W")

Getting AAPL data for 2024
Getting AAPL data for 2025
Getting TSLA data for 2024
Getting TSLA data for 2025
Getting AMZN data for 2024
Getting AMZN data for 2025


In [7]:
def getNewsData(API_KEY, tickers, time_from, time_to, count=0):
    tickerString = ""
    topicString = ""
    fromString = ""
    toString = ""

    count = checkRequests(count)

    if tickers != None:
        tickerString = f"&tickers={tickers}"
    if time_from != None:
        fromString = f"&time_from={time_from}"
    if time_to != None:
        toString = f"&time_to={time_to}"
    
    URL = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT{tickerString}{fromString}{toString}&apikey={API_KEY}&limit=1000&sort=EARLIEST"
    response = requests.get(URL)
    data = response.json()
    data = data['feed']
    df = pd.DataFrame.from_dict(data)
    return df, count

def checkRequests(count):
    if count == 75:
        print("API requests reached for min... Waiting")
        time.sleep(62)
        count = 0
    count += 1
    return count

def getNewsForPeriod(API_KEY, tickers, dateStart, dateEnd, count):
    start = datetime.strptime(dateStart, '%Y%m%d')
    end = datetime.strptime(dateEnd, '%Y%m%d')
    dfList = []

    current = start
    print(f"Getting data for {tickers} from {start.date()} to {end.date()}")
    while current < end:
        time_from = current.strftime('%Y%m%dT0001')
        if (current + relativedelta(weeks=3)) > end:
            current = end
        else:
            current += relativedelta(weeks=3)
        time_to = current.strftime('%Y%m%dT2359')
        df, count = getNewsData(API_KEY=API_KEY, tickers=tickers, time_from=time_from, time_to=time_to, count=count)
        dfList.append(df)
    full_df = pd.concat(dfList, ignore_index=True)
    return full_df, count

In [8]:
API_KEY = "5HUC90FRQ4H9PK0Q"

aapl_news, count = getNewsForPeriod(API_KEY=API_KEY, tickers="AAPL", dateStart="20240101", dateEnd="20250601", count=count)
tsla_news, count = getNewsForPeriod(API_KEY=API_KEY, tickers="TSLA", dateStart="20240101", dateEnd="20250601", count=count)
amzn_news, count = getNewsForPeriod(API_KEY=API_KEY, tickers="AMZN", dateStart="20240101", dateEnd="20250601", count=count)
btc_news, count = getNewsForPeriod(API_KEY=API_KEY, tickers="CRYPTO:BTC", dateStart="20240101", dateEnd="20250601", count=count)
eth_news, count = getNewsForPeriod(API_KEY=API_KEY, tickers="CRYPTO:ETH", dateStart="20240101", dateEnd="20250601", count=count)

Getting data for AAPL from 2024-01-01 to 2025-06-01
API requests reached for min... Waiting
Getting data for TSLA from 2024-01-01 to 2025-06-01
Getting data for AMZN from 2024-01-01 to 2025-06-01
Getting data for CRYPTO:BTC from 2024-01-01 to 2025-06-01
API requests reached for min... Waiting
Getting data for CRYPTO:ETH from 2024-01-01 to 2025-06-01


In [9]:
import ast
import re

all_topics = []

def clean_data(df, tickerName=''):
    copy_df = df.copy()
    copy_df = copy_df[['title', 'time_published', 'summary', 'topics', 'overall_sentiment_score', 'ticker_sentiment']]
    
    for topic_list in copy_df['topics']:
        if isinstance(topic_list, str):
            topic_list = ast.literal_eval(topic_list)
        for topic_dict in topic_list:
            if topic_dict['topic'] not in all_topics:
                all_topics.append(topic_dict['topic'])

    copy_df[['ticker_relevance', 'ticker_sentiment']] = copy_df['ticker_sentiment'].apply(lambda row: add_ticker_info(row, tickerName))
    weighted_df = copy_df['topics'].apply(add_topic_relevance)
    weighted_df = weighted_df.rename(columns=lambda x: f"topic_{x}")
    copy_df = pd.concat([copy_df, weighted_df], axis = 1)
    copy_df = copy_df.drop(columns=['topics'])

    copy_df['time_published'] = pd.to_datetime(copy_df['time_published'], format='%Y%m%dT%H%M%S')
    copy_df['time_published'] = copy_df['time_published'].dt.strftime('%Y-%m-%d %H:%M:%S')
    copy_df = copy_df.rename(columns={'time_published': 'timestamp'})
    copy_df = copy_df.drop_duplicates()
    copy_df = copy_df.dropna()
    copy_df['title'] = copy_df['title'].apply(clean_text)
    copy_df['summary'] = copy_df['summary'].apply(clean_text)
    copy_df = copy_df.set_index('timestamp')
    return copy_df

def add_topic_relevance(topics_row):
    if isinstance(topics_row, str):
        topics_list = ast.literal_eval(topics_row)
    else:
        topics_list = topics_row

    topic_weights = {topic: 0.0 for topic in all_topics}

    for topic_info in topics_list:
        topic_name = topic_info['topic']
        relevance = float(topic_info['relevance_score'])
        if topic_name in topic_weights:
            topic_weights[topic_name] = relevance

    return pd.Series(topic_weights)


def add_ticker_info(ticker_row, tickerName):
    if isinstance(ticker_row, str):
        ticker_list = ast.literal_eval(ticker_row)
    else:
        ticker_list = ticker_row

    for ticker_info in ticker_list:
        if ticker_info['ticker'] == tickerName:
            relevance = float(ticker_info['relevance_score'])
            sentiment = float(ticker_info['ticker_sentiment_score'])
            return pd.Series([relevance, sentiment])
    
    return pd.Series([0.0, 0.0])

def clean_text(text):
    if pd.isnull(text):
        return ""
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [10]:
aapl_news = clean_data(aapl_news, 'AAPL')
tsla_news = clean_data(tsla_news, 'TSLA')
amzn_news = clean_data(amzn_news, 'AMZN')
btc_news = clean_data(btc_news, 'CRYPTO:BTC')
eth_news = clean_data(eth_news, 'CRYPTO:ETH')

In [11]:
import ta, ta.trend, ta.momentum

def add_ta_features(df, timeframe,rsi_window=14, sma_windows=(5, 20)):
    df = df.copy()
    
    rsi = ta.momentum.RSIIndicator(close=df["close"], window=rsi_window)
    df["rsi_14"] = rsi.rsi()
    
    macd = ta.trend.MACD(close=df["close"])
    df["macd"] = macd.macd()
    df["macd_signal"] = macd.macd_signal()

    sma_short = ta.trend.SMAIndicator(close=df["close"], window=sma_windows[0])
    sma_long = ta.trend.SMAIndicator(close=df["close"], window=sma_windows[1])
    
    df[f"sma_{sma_windows[0]}"] = sma_short.sma_indicator()
    df[f"sma_{sma_windows[1]}"] = sma_long.sma_indicator()

    df["return"] = df["close"].pct_change(periods=timeframe)

    df["future_return"] = df["close"].shift(-1) / df["close"] - 1
    df["target"] = (df["future_return"] > 0).astype(int)
    df = df.dropna()
    return df

def aggregate_and_forwardfill_news(news_df, timeframe='1H'):
    news_df = news_df.reset_index()
    news_df['timestamp'] = pd.to_datetime(news_df['timestamp'])
    # news_df.set_index("timestamp", inplace=True)
    numeric_cols = news_df.select_dtypes(include='number').columns.tolist()
    numeric_cols.append('timestamp')
    news_df = news_df[numeric_cols]
    news_df = news_df.set_index('timestamp')
    news_aggregated = news_df.resample(timeframe).mean()
    sentiment_columns = news_aggregated.columns
    news_aggregated[sentiment_columns] = news_aggregated[sentiment_columns].ffill()
    news_aggregated[sentiment_columns] = news_aggregated[sentiment_columns].fillna(0)
    news_aggregated = news_aggregated.reset_index()
    # news_df.reset_index(inplace=True)
    return news_aggregated

def merge_df(df1, df2):
    df1['timestamp'] = pd.to_datetime(df1['timestamp'])
    df2['timestamp'] = pd.to_datetime(df2['timestamp'])

    df1 = df1.set_index('timestamp')
    df2 = df2.set_index('timestamp')

    merged_df = df1.join(df2, how='left')
    merged_df = merged_df.reset_index()
    sentiment_cols = df2.columns
    merged_df[sentiment_cols] = merged_df[sentiment_cols].fillna(0)
    return merged_df

def keep_news(df):
    news_df = df[df['ticker_sentiment'] != 0.0]
    return news_df

In [12]:
btc_hourly = add_ta_features(btc_1h, 1)
btc_4hourly = add_ta_features(btc_4h, 1)
btc_daily = add_ta_features(btc_1d, 1)
btc_weekly = add_ta_features(btc_1w, 1)

btc_news_hourly = aggregate_and_forwardfill_news(btc_news, '1h')
btc_news_4hourly = aggregate_and_forwardfill_news(btc_news, '4h')
btc_news_daily = aggregate_and_forwardfill_news(btc_news, '1d')
btc_news_weekly = aggregate_and_forwardfill_news(btc_news, '1W')

eth_hourly = add_ta_features(eth_1h, 1)
eth_4hourly = add_ta_features(eth_4h, 1)
eth_daily = add_ta_features(eth_1d, 1)
eth_weekly = add_ta_features(eth_1w, 1)

eth_news_hourly = aggregate_and_forwardfill_news(eth_news, '1h')
eth_news_4hourly = aggregate_and_forwardfill_news(eth_news, '4h')
eth_news_daily = aggregate_and_forwardfill_news(eth_news, '1d')
eth_news_weekly = aggregate_and_forwardfill_news(eth_news, '1W')

aapl_hourly = add_ta_features(aapl_1h, 1)
aapl_4hourly = add_ta_features(aapl_4h, 1)
aapl_daily = add_ta_features(aapl_1d, 1)
aapl_weekly = add_ta_features(aapl_1w, 1)

aapl_news_hourly = aggregate_and_forwardfill_news(aapl_news, '1h')
aapl_news_4hourly = aggregate_and_forwardfill_news(aapl_news, '4h')
aapl_news_daily = aggregate_and_forwardfill_news(aapl_news, '1d')
aapl_news_weekly = aggregate_and_forwardfill_news(aapl_news, 'W-FRI')

tsla_hourly = add_ta_features(tsla_1h, 1)
tsla_4hourly = add_ta_features(tsla_4h, 1)
tsla_daily = add_ta_features(tsla_1d, 1)
tsla_weekly = add_ta_features(tsla_1w, 1)

tsla_news_hourly = aggregate_and_forwardfill_news(tsla_news, '1h')
tsla_news_4hourly = aggregate_and_forwardfill_news(tsla_news, '4h')
tsla_news_daily = aggregate_and_forwardfill_news(tsla_news, '1d')
tsla_news_weekly = aggregate_and_forwardfill_news(tsla_news, 'W-FRI')

amzn_hourly = add_ta_features(amzn_1h, 1)
amzn_4hourly = add_ta_features(amzn_4h, 1)
amzn_daily = add_ta_features(amzn_1d, 1)
amzn_weekly = add_ta_features(amzn_1w, 1)

amzn_news_hourly = aggregate_and_forwardfill_news(amzn_news, '1h')
amzn_news_4hourly = aggregate_and_forwardfill_news(amzn_news, '4h')
amzn_news_daily = aggregate_and_forwardfill_news(amzn_news, '1d')
amzn_news_weekly = aggregate_and_forwardfill_news(amzn_news, 'W-FRI')

In [13]:
merged_btc_hourly = merge_df(btc_hourly, btc_news_hourly)
merged_btc_4hourly = merge_df(btc_4hourly, btc_news_4hourly)
merged_btc_daily = merge_df(btc_daily, btc_news_daily)
merged_btc_weekly = merge_df(btc_weekly, btc_news_weekly)

merged_eth_hourly = merge_df(eth_hourly, eth_news_hourly)
merged_eth_4hourly = merge_df(eth_4hourly, eth_news_4hourly)
merged_eth_daily = merge_df(eth_daily, eth_news_daily)
merged_eth_weekly = merge_df(eth_weekly, eth_news_weekly)

merged_aapl_hourly = merge_df(aapl_hourly, aapl_news_hourly)
merged_aapl_4hourly = merge_df(aapl_4hourly, aapl_news_4hourly)
merged_aapl_daily = merge_df(aapl_daily, aapl_news_daily)
merged_aapl_weekly = merge_df(aapl_weekly, aapl_news_weekly)

merged_tsla_hourly = merge_df(tsla_hourly, tsla_news_hourly)
merged_tsla_4hourly = merge_df(tsla_4hourly, tsla_news_4hourly)
merged_tsla_daily = merge_df(tsla_daily, tsla_news_daily)
merged_tsla_weekly = merge_df(tsla_weekly, tsla_news_weekly)

merged_amzn_hourly = merge_df(amzn_hourly, amzn_news_hourly)
merged_amzn_4hourly = merge_df(amzn_4hourly, amzn_news_4hourly)
merged_amzn_daily = merge_df(amzn_daily, amzn_news_daily)
merged_amzn_weekly = merge_df(amzn_weekly, amzn_news_weekly)

In [14]:
# Saving for BTC
merged_btc_hourly.to_csv("app/datasets/btc_hourly.csv")
merged_btc_4hourly.to_csv("app/datasets/btc_4hourly.csv")
merged_btc_daily.to_csv("app/datasets/btc_daily.csv")
merged_btc_weekly.to_csv("app/datasets/btc_weekly.csv")

# Saving for ETH
merged_eth_hourly.to_csv("app/datasets/eth_hourly.csv")
merged_eth_4hourly.to_csv("app/datasets/eth_4hourly.csv")
merged_eth_daily.to_csv("app/datasets/eth_daily.csv")
merged_eth_weekly.to_csv("app/datasets/eth_weekly.csv")

# Saving for AAPL
merged_aapl_hourly.to_csv("app/datasets/aapl_hourly.csv")
merged_aapl_4hourly.to_csv("app/datasets/aapl_4hourly.csv")
merged_aapl_daily.to_csv("app/datasets/aapl_daily.csv")
merged_aapl_weekly.to_csv("app/datasets/aapl_weekly.csv")

# Saving for TSLA
merged_tsla_hourly.to_csv("app/datasets/tsla_hourly.csv")
merged_tsla_4hourly.to_csv("app/datasets/tsla_4hourly.csv")
merged_tsla_daily.to_csv("app/datasets/tsla_daily.csv")
merged_tsla_weekly.to_csv("app/datasets/tsla_weekly.csv")

# Saving for AMZN
merged_amzn_hourly.to_csv("app/datasets/amzn_hourly.csv")
merged_amzn_4hourly.to_csv("app/datasets/amzn_4hourly.csv")
merged_amzn_daily.to_csv("app/datasets/amzn_daily.csv")
merged_amzn_weekly.to_csv("app/datasets/amzn_weekly.csv")